# **Predictive Default Risk Assessor V.01**

In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
with open("hello_credit_config.json", "r") as f:
    config = json.load(f)

# 

In [19]:
class CreditRatingCalculator:
    def __init__(self, metrics, credit_ratings):
        self.metrics = metrics
        self.credit_ratings = credit_ratings
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        return next(rating for rating, threshold in self.credit_ratings.items() if weighted_score <= threshold)

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [20]:
model_metrics = [
    'oper_margin', 'tot_debt_to_tot_eqy', 'tot_debt_to_ebitda',
    'ebitda_to_tot_int_exp', 'return_on_asset', 'asset_turnover',
]

features = pd.read_excel("research/metrics_full.xlsx", index_col=0).T
#features = pd.read_excel("dataset/ibx_jalsh_features.xlsx", index_col=0, )
features.columns = features.columns.str.lower()
features = features[model_metrics]

ratings = pd.read_excel("dataset/ibx_jalsh_credit_ratings.xlsx", index_col=0)

In [51]:
company = "CLS SJ Equity" 

In [103]:
ratios = features.loc[company].to_dict()
model = CreditRatingCalculator(config['credit_model_config'], config['credit_rating_scale'])
model.calculate_credit_rating(ratios)
print(company)
print(f"Model Inputs:")
display(ratios)
print("")
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

CLS SJ Equity
Model Inputs:


{'oper_margin': 6.531625438897736,
 'tot_debt_to_tot_eqy': 29.34166481789502,
 'tot_debt_to_ebitda': 0.4861134330769339,
 'ebitda_to_tot_int_exp': 19.8299600262328,
 'return_on_asset': 11.53146781672658,
 'asset_turnover': 2.719620868494022}


Class Scoring: {'profitability': 8.0, 'leverage_coverage': 3.1, 'efficiency': 2.0}
Credit Score: 4.405
Credit Rating: A


In [86]:
df = pd.read_excel("dataset/ibx_jalsh_raw_data.xlsx", index_col=0)

In [91]:
rc = df[["model_credit_ratings", "RSK_BB_ISSUER_DEFAULT", "RTG_MOODY_LONG_TERM", "RTG_SP_LT_LC_ISSUER_CREDIT"]]

In [95]:
mappings = config['credit_rating_mappings']

In [96]:
# Create a reverse mapping dictionary
reverse_mappings = {value: key for key, values in mappings.items() for value in values}

In [98]:
rc

,model_credit_ratings,RSK_BB_ISSUER_DEFAULT,RTG_MOODY_LONG_TERM,RTG_SP_LT_LC_ISSUER_CREDIT
ABG SJ Equity,Baa,IG6,NaN,NaN
ADH SJ Equity,A,IG1,NaN,NaN
AEL SJ Equity,Ba,IG7,NaN,NaN
AFE SJ Equity,Ba,IG2,NaN,NaN
AFH SJ Equity,Ba,IG5,NaN,NaN
...,...,...,...,...
TSG SJ Equity,Ba,IG5,NaN,NaN
VKE SJ Equity,Baa,IG4,NaN,NaN
VOD SJ Equity,A,IG2,NaN,NaN
WBO SJ Equity,A,IG1,NaN,NaN


In [99]:
rc['RSK_BB_ISSUER_DEFAULT'] = rc['RSK_BB_ISSUER_DEFAULT'].map(lambda x: reverse_mappings.get(x, x))
rc['RTG_MOODY_LONG_TERM'] = rc['RTG_MOODY_LONG_TERM'].map(lambda x: reverse_mappings.get(x, x))
rc['RTG_SP_LT_LC_ISSUER_CREDIT'] = rc['RTG_SP_LT_LC_ISSUER_CREDIT'].map(lambda x: reverse_mappings.get(x, x))

C:\Users\Thabang Ndhlovu\AppData\Local\Temp\ipykernel_12776\2534808706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc['RSK_BB_ISSUER_DEFAULT'] = rc['RSK_BB_ISSUER_DEFAULT'].map(lambda x: reverse_mappings.get(x, x))
C:\Users\Thabang Ndhlovu\AppData\Local\Temp\ipykernel_12776\2534808706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc['RTG_MOODY_LONG_TERM'] = rc['RTG_MOODY_LONG_TERM'].map(lambda x: reverse_mappings.get(x, x))
C:\Users\Thabang Ndhlovu\AppData\Local\Temp\ipykernel_12776\2534808706

In [101]:
rc.to_excel("rc.xlsx")

In [ ]:

# study = {}
# model = CreditRatingCalculator(config['credit_model_config'], config['credit_rating_scale'])

# for company in features.index:
#     ratios = features.loc[company].to_dict()
#     model.calculate_credit_rating(ratios)
    
    
#     study[company] = {
#         "model_credit_ratings": model.credit_rating,
#         "model_credit_scores": model.credit_score,
#         "model_class_scores": model.scores,
#     }
    